# Market Microstructure Analysis — SPY

Main analysis notebook. We look at:
1. How microstructure features change between normal and crisis markets
2. Order book dynamics on selected days
3. Execution strategy performance across different regimes

Data: SPY 1-minute bars, Jan-Apr 2020.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import sys
sys.path.append('..')

from src.data_collection import load_bars, clean_bars, add_derived_columns, get_daily_summary
from src.features import compute_all_features
from src.orderbook import reconstruct_book, book_summary_by_day
from src.execution import compare_strategies, compare_across_days
from src.visualization import (
    plot_book_dynamics, plot_spread_comparison,
    plot_execution_comparison, plot_execution_fills,
    plot_feature_regime_comparison, plot_price_volume,
    plot_feature_correlation, plot_daily_feature_timeseries
)

plt.rcParams['figure.dpi'] = 100

# our 5 analysis days
ANALYSIS_DATES = [
    datetime.date(2020, 1, 16),   # quiet
    datetime.date(2020, 1, 24),   # moderate
    datetime.date(2020, 2, 27),   # early crisis
    datetime.date(2020, 3, 12),   # peak chaos
    datetime.date(2020, 3, 24),   # recovery
]

In [ ]:
# load and process everything
bars = load_bars()
bars = clean_bars(bars)
bars = add_derived_columns(bars)

# label periods
bars['period'] = 'normal'
bars.loc[bars['date'] >= pd.Timestamp('2020-02-24').date(), 'period'] = 'crisis'

# compute features
bars = compute_all_features(bars)

# reconstruct order book
bars = reconstruct_book(bars)

print(f"\nReady: {len(bars):,} bars with {len(bars.columns)} columns")

---
## 1. Microstructure Features: Normal vs Crisis

How did market quality change when COVID hit?

In [ ]:
# spread widening
plot_feature_regime_comparison(bars, 'cs_spread', 'Corwin-Schultz Spread')
plt.show()

In [ ]:
# volatility regime shift
plot_feature_regime_comparison(bars, 'realized_vol', 'Realized Volatility (20-bar)')
plt.show()

In [ ]:
# order flow imbalance
plot_feature_regime_comparison(bars, 'ofi', 'Order Flow Imbalance')
plt.show()

In [ ]:
# return autocorrelation — does mean reversion break down in crisis?
plot_feature_regime_comparison(bars, 'ret_autocorr', 'Return Autocorrelation (20-bar)')
plt.show()

In [ ]:
# aggregate comparison table
feature_cols = ['cs_spread', 'hl_spread', 'realized_vol', 'parkinson_vol',
                'ofi', 'amihud', 'ret_autocorr', 'spread_bps']

regime_comparison = bars.groupby('period')[feature_cols].agg(['mean', 'std']).round(6)
regime_comparison

### Feature Correlations

Do relationships between features change when markets shift to crisis mode?
In normal markets, spread and volatility should move together. In a crisis,
the correlation structure can break down or intensify.

In [ ]:
# correlation heatmap — normal vs crisis
corr_features = ['cs_spread', 'hl_spread', 'realized_vol', 'parkinson_vol',
                 'ofi', 'amihud', 'ret_autocorr', 'book_imbalance']

plot_feature_correlation(bars, corr_features)
plt.show()

In [ ]:
# daily spread and vol over the full sample — shows the transition clearly
daily_features = bars.groupby('date').agg(
    cs_spread=('cs_spread', 'mean'),
    realized_vol=('realized_vol', 'mean'),
    spread_bps=('spread_bps', 'mean'),
    avg_volume=('volume', 'mean'),
).reset_index()

plot_daily_feature_timeseries(daily_features, 'spread_bps', 'Avg Spread (bps)')
plt.title('Daily Average Spread — Liquidity Evaporated During COVID')
plt.show()

plot_daily_feature_timeseries(daily_features, 'realized_vol', 'Realized Volatility')
plt.title('Daily Realized Volatility — Order of Magnitude Increase')
plt.show()

---
## 2. Order Book Dynamics

How does the estimated order book behave on our selected days?

In [ ]:
# spread across all days
book_summary = book_summary_by_day(bars)
plot_spread_comparison(book_summary, ANALYSIS_DATES)
plt.show()

In [ ]:
# book dynamics: quiet day
day = bars[bars['date'] == ANALYSIS_DATES[0]]
plot_book_dynamics(day, 'Quiet Day — Jan 16')
plt.show()

In [ ]:
# book dynamics: peak chaos
day = bars[bars['date'] == ANALYSIS_DATES[3]]
plot_book_dynamics(day, 'Peak Chaos — Mar 12')
plt.show()

In [ ]:
# book dynamics: recovery
day = bars[bars['date'] == ANALYSIS_DATES[4]]
plot_book_dynamics(day, 'Recovery — Mar 24')
plt.show()

---
## 3. Execution Strategy Comparison

Simulating a 100k share buy order across strategies.
How does the optimal strategy change with market conditions?

In [ ]:
# run all strategies on each analysis day
print("Running execution simulations...")
all_results = {}
all_fills = {}

for date in ANALYSIS_DATES:
    day = bars[bars['date'] == date].reset_index(drop=True)
    results_df, fills_dict = compare_strategies(day)
    all_results[date] = results_df
    all_fills[date] = fills_dict
    print(f"  {date} done")

print("Done.")

In [ ]:
# show results for each day
for date in ANALYSIS_DATES:
    print(f"\n{'='*60}")
    print(f"  {date}")
    print(f"{'='*60}")
    display(all_results[date][['avg_exec_price', 'benchmark_vwap',
                               'slippage_vs_vwap_bps', 'slippage_vs_arrival_bps']].round(4))

In [ ]:
# visual comparison: quiet day
plot_execution_comparison(all_results[ANALYSIS_DATES[0]], ANALYSIS_DATES[0])
plt.show()

plot_execution_fills(all_fills[ANALYSIS_DATES[0]],
                     bars[bars['date'] == ANALYSIS_DATES[0]].reset_index(drop=True),
                     ANALYSIS_DATES[0])
plt.show()

In [ ]:
# visual comparison: peak chaos
plot_execution_comparison(all_results[ANALYSIS_DATES[3]], ANALYSIS_DATES[3])
plt.show()

plot_execution_fills(all_fills[ANALYSIS_DATES[3]],
                     bars[bars['date'] == ANALYSIS_DATES[3]].reset_index(drop=True),
                     ANALYSIS_DATES[3])
plt.show()

In [ ]:
# visual comparison: recovery
plot_execution_comparison(all_results[ANALYSIS_DATES[4]], ANALYSIS_DATES[4])
plt.show()

plot_execution_fills(all_fills[ANALYSIS_DATES[4]],
                     bars[bars['date'] == ANALYSIS_DATES[4]].reset_index(drop=True),
                     ANALYSIS_DATES[4])
plt.show()

---
## 4. Summary of slippage across all days

How does slippage scale with volatility?

In [ ]:
# build summary table
summary_rows = []
daily = get_daily_summary(bars)

for date in ANALYSIS_DATES:
    day_info = daily[daily['date'] == date].iloc[0]
    for strategy in all_results[date].index:
        row = {
            'date': date,
            'strategy': strategy,
            'day_range_pct': day_info['day_range_pct'],
            'total_volume_M': day_info['total_volume'] / 1e6,
            'slippage_bps': all_results[date].loc[strategy, 'slippage_vs_vwap_bps'],
        }
        summary_rows.append(row)

slippage_summary = pd.DataFrame(summary_rows)
slippage_summary

In [ ]:
# plot: slippage vs volatility for each strategy
fig, ax = plt.subplots(figsize=(10, 6))

for strategy in ['VWAP', 'TWAP', 'Aggressive', 'Passive']:
    subset = slippage_summary[slippage_summary['strategy'] == strategy]
    ax.scatter(subset['day_range_pct'], subset['slippage_bps'].abs(),
              label=strategy, s=80, alpha=0.8)

ax.set_xlabel('Day Range (%)')
ax.set_ylabel('|Slippage vs VWAP| (bps)')
ax.set_title('Execution Slippage Increases with Volatility')
ax.legend()
plt.tight_layout()
plt.show()

---
## Key Takeaways

1. **Spreads widened 10-20x during the crisis** — from ~2-3 bps in January to 30-40+ bps in mid-March. This directly increases execution costs.

2. **Volatility regime shift was dramatic** — realized vol increased by an order of magnitude, and the intraday profile changed shape (less of a clean U-shape, more uniformly elevated).

3. **Order flow imbalance became more extreme** — the book was consistently skewed during the crash, indicating one-sided markets.

4. **VWAP is the most consistent strategy** — by definition it tracks the benchmark, but the fact that other strategies show increasing deviation during volatile markets highlights why VWAP is the institutional standard.

5. **Passive strategies are risky in trending markets** — the passive strategy got lucky on big down days (bought cheaper at the end), but this is hindsight bias. In a real V-shaped reversal, you'd get crushed.

6. **Aggressive execution has a hidden cost** — the market impact model shows that front-loading orders costs more per share due to information leakage and temporary impact.

7. **Depth evaporated during the crisis** — estimated book depth dropped as market makers widened quotes and reduced size, consistent with the well-documented liquidity withdrawal during COVID.